In [ ]:
# Class Distribution Analysis 
class_distribution = Counter(helpfulness_ratings)
total_data = len(helpfulness_ratings)


for rating, count in class_distribution.items():
    percentage = (count / total_data) * 100
    print(f"Class {rating}: {count} ({percentage:.2f}%)")


plt.figure(figsize=(8, 5))
plt.bar(class_distribution.keys(), class_distribution.values())
plt.xlabel('Helpfulness Label')
plt.ylabel('Count')
plt.title('Class Distribution in Full Dataset')
plt.show()

In [ ]:
# Sort to ensure consistent ordering
unique_labels = sorted(list(set(helpfulness_ratings)))
unique_one_hot = np.diag(np.ones(len(unique_labels)))


labels_train = [helpfulness_ratings[i] for i in train_ints]
labels_test = [helpfulness_ratings[i] for i in test_ints]
labels_dev = [helpfulness_ratings[i] for i in final_test_ints]

# Make sure you're using the right label variables
y_train = np.array([list(unique_one_hot[k]) for k in [unique_labels.index(x) for x in labels_train]]).T
y_test = np.array([list(unique_one_hot[k]) for k in [unique_labels.index(x) for x in labels_test]]).T
y_dev = np.array([list(unique_one_hot[k]) for k in [unique_labels.index(x) for x in labels_dev]]).T

In [ ]:
# Dataset example 
train_distribution = Counter(labels_train)
print(train_distribution)
test_distribution = Counter(labels_test)
print(test_distribution)
dev_distribution = Counter(labels_dev)
print(dev_distribution)


In [ ]:
def LogisticRegression_Softmax(x_train_data, y_train_data,x_test,y_test,lr =0.1,n_iters=1000,random_seed=42):
    np.random.seed(random_seed)
    

    num_features = x_train_data.shape[1]
    num_classes = y_train_data.shape[0]
    num_samples = x_train_data.shape[0]

    # Initialize weights with correct dimensions: (num_features, num_classes)
    weights = np.random.rand(num_features, num_classes) # (8000,3)
    bias = np.zeros(num_classes) # (3,)
    
    logistic_loss=[]



    # # x and y now refer to the training data for intent classification
    x_train_data = x_train_data  # Shape (num_samples, num_features) (21928, 8000)
    y_train_targets = y_train_data.T # Shape (num_samples, num_classes) (3, 21928)
    # print(x_train_data.shape)
    # print(y_train_targets.shape)




    for i in range(n_iters):

        # z = x.dot(weights) expects x to be (num_samples, num_features) and weights to be (num_features, num_classes)
        z= x_train_data.dot(weights) + bias
        
        # Softmax 
        exp_z = np.exp(z)
        q = exp_z / np.sum(exp_z, axis=1, keepdims=True)
        # exp_z = np.exp(z)
        # q = exp_z / np.sum(exp_z, axis=1, keepdims=True)
        
        
        # Calculate loss using y_train_targets
        eps = 1e-12
        # Softmax cross entropy loss with clipping to prevent log(0)
        loss = (-np.sum(y_train_targets*np.log2(np.clip(q, eps, 1.0)))) / num_samples # avoid log(0)
    
        logistic_loss.append(loss)

        # dw = x.T.dot((q-y)) expects x.T to be (num_features, num_samples) and (q-y) to be (num_samples, num_classes)
        # Here x_train_data.T is (num_features, num_samples)
        db = np.sum((q-y_train_targets), axis=0)/num_samples
        dw=x_train_data.T.dot((q-y_train_targets))/num_samples
        weights=(weights - (dw*lr))
        bias=(bias - (db*lr))
    result = LogReg_Softmax_Test(weights,bias,x_test,y_test)
    return weights,bias,result,logistic_loss

In [ ]:
def LogReg_Softmax_Test(weights,bias,x_dataset,y_dataset):
    # Forward pass on test data
    z_test = x_dataset.dot(weights) + bias
    # Perform theSoftmax
    exp_z_test = np.exp(z_test)
    q_test = exp_z_test / np.sum(exp_z_test, axis=1, keepdims=True)
    # Get predictions, basically get the class with highest probability
    y_pred_test = np.argmax(q_test, axis=1)  # Shape: (7310,)

    # Get true labels
    y_test_targets = y_dataset.T  
    y_true_test = np.argmax(y_test_targets, axis=1)  # Shape: (7310,)

    # Calculate accuracy
    accuracy_test = np.mean(y_pred_test == y_true_test)
    print(f"Test Accuracy: {accuracy_test * 100:.2f}%")
    TP = []
    FP = []
    FN = []
    for j in range(3):
        TP.append(np.sum(np.array([int(s == j and y_true_test[i] == j) for i,s in enumerate(y_pred_test)])))
        FP.append(np.sum(np.array([int(s == j and y_true_test[i] != j) for i,s in enumerate(y_pred_test)])))
        FN.append(np.sum(np.array([int(s != j and y_true_test[i] == j) for i,s in enumerate(y_pred_test)])))
    
    precision = np.array(TP)/(np.array(TP)+np.array(FP))
    recall = np.array(TP)/(np.array(TP)+np.array(FN))
    f1 = 2*precision*recall/(precision+recall)
    print(precision)
    print(recall)
    print(f1)

In [ ]:

LogReg_models_softmax = {}
for lr in learning_rates:
    print("Learning rate: ",lr)
    weights,bias,result,loss = LogisticRegression_Softmax(M_train,y_train,M_test,y_test,lr=lr) 
    print("-----")
    LogReg_models_softmax[lr] = (weights,bias,result,loss)
    


In [ ]:
def MLP_softmax(x_train,y_train,x_test,y_test,lr=0.1,hidden_size=16,n_iters=1000,):

    # Add more layers for hte multilayer perceptron
    num_features= x_train.shape[1]
    num_classes = 3

    helpfulness_labels = y_train # 3,2198
    np.random.seed(1)

    # Intitiliasation method based on Xavier's proposed method 
    # https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf#page=4.98
    limit_1 = np.sqrt(6 / (num_features ))
    weights_0_1 = np.random.uniform(-limit_1, limit_1, (num_features, hidden_size))
    limit_2 = np.sqrt(6 / (hidden_size ))
    weights_1_2 = np.random.uniform(-limit_2, limit_2, (hidden_size,num_classes))
    y_train_target = y_train.T

    loss_history = []
    
    num_samples = x_train.shape[0] # Number of training samples

    for iteration in range(n_iters):

        layer_2_error = 0
        layer_0 = x_train

        ## Add forward pass
        layer_1 = np.maximum(np.dot(layer_0,weights_0_1),0)
        layer_2 = np.dot(layer_1,weights_1_2)
    


        # Then apply sigmoid
        # layer_2_s = 1/(1+np.exp(-layer_2))
        exp_z = np.exp(layer_2)
        layer_2_s = exp_z / np.sum(exp_z, axis=1, keepdims=True)
        q = layer_2_s


    
        eps = 1e-8
         # Softmax cross entropy loss with clipping to prevent log(0)
        loss = -(np.sum(y_train.T*np.log2(np.clip(q, eps, 1.0))))/num_samples  # avoid log(0)
        loss_history.append(loss)
        

        ## Add backward pass and update weights
        layer_2_diff = (layer_2_s - y_train_target)

        z1 = np.dot(layer_0, weights_0_1)
        relu_grad = (z1 > 0).astype(float)



        hidden_delta = np.dot(layer_2_diff, weights_1_2.T) * relu_grad

        # Normalize weight updates by N
        weights_1_2 -= lr   * (np.dot(layer_1.T, layer_2_diff) /num_samples)
        weights_0_1 -= lr * (np.dot(layer_0.T, hidden_delta) / num_samples)
    result = MLP_softmax_test(weights_0_1,weights_1_2,x_test,y_test)    
    return weights_0_1, weights_1_2,result, loss_history

def MLP_softmax_test(weights_0_1, weights_1_2, x_dataset,y_dataset):
    layer_0_test = x_dataset    
    layer_1_test = np.maximum(np.dot(layer_0_test, weights_0_1), 0)  # ReLU activation
    layer_2_test = np.dot(layer_1_test, weights_1_2)

    # Apply softmax
    exp_z_test = np.exp(layer_2_test)
    layer_2_s_test = exp_z_test / np.sum(exp_z_test, axis=1, keepdims=True)

    # Get predictions (class with highest probability)
    y_pred_test = np.argmax(layer_2_s_test, axis=1)

    # Get true labels
    y_true_test = np.argmax(y_dataset.T, axis=1)

    # Calculate accuracy
    accuracy_test = np.mean(y_pred_test == y_true_test)
    print(f"Test Accuracy: {accuracy_test * 100:.2f}%")
    TP = []
    FP = []
    FN = []
    for j in range(3):
        TP.append(np.sum(np.array([int(s == j and y_true_test[i] == j) for i,s in enumerate(y_pred_test)])))
        FP.append(np.sum(np.array([int(s == j and y_true_test[i] != j) for i,s in enumerate(y_pred_test)])))
        FN.append(np.sum(np.array([int(s != j and y_true_test[i] == j) for i,s in enumerate(y_pred_test)])))
    
    precision = np.array(TP)/(np.array(TP)+np.array(FP))
    recall = np.array(TP)/(np.array(TP)+np.array(FN))
    f1 = 2*precision*recall/(precision+recall)
    print(precision)
    print(recall)
    print(f1)

In [ ]:
MLP_models_softmax = {}
for lr in learning_rates:
    print("Learning rate: ",lr)
    weights,bias,result,loss = MLP_softmax(M_train,y_train,M_test,y_test,lr=lr) 
    print("-----")
    MLP_models_softmax[lr] = (weights,bias,result,loss)
